<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Tracing and Evaluating a DSPy Application</h1>

DSPy is a framework for automatically prompting and fine-tuning language models. It provides:

- Composable and declarative APIs that allow developers to describe the architecture of their LLM application in the form of a "module" (inspired by PyTorch's `nn.Module`),
- Compilers known as "teleprompters" that optimize a user-defined module for a particular task. The term "teleprompter" is meant to evoke "prompting at a distance," and could involve selecting few-shot examples, generating prompts, or fine-tuning language models.

Phoenix makes your DSPy applications *observable* by visualizing the underlying structure of each call to your compiled DSPy module and surfacing problematic spans of execution based on latency, token count, or other evaluation metrics.

In this tutorial, you will:
- Build and compile a  DSPy module that uses retrieval-augmented generation to answer questions over the [HotpotQA dataset](https://hotpotqa.github.io/wiki-readme.html),
- Instrument your application using [OpenInference](https://github.com/Arize-ai/openinference), and open standard for recording your LLM telemetry data,
- Inspect the traces and spans of your application to understand the inner works of a DSPy forward pass.

ℹ️ This notebook requires an OpenAI API key.


## 1. Install Dependencies and Import Libraries

Install Phoenix, DSPy, and other dependencies.

In [1]:
# pip install openinference-instrumentation-dspy dspy-ai arize-phoenix opentelemetry-sdk opentelemetry-exporter-otlp

⚠️ DSPy conflicts with the default version of the `regex` module that comes pre-installed on Google Colab. If you are running this notebook in Google Colab, you will likely need to restart the kernel after running the installation step above and before proceeding to the rest of the notebook, otherwise, your instrumentation will fail.

Import libraries.

In [2]:
import os
# from getpass import getpass

import dspy
# import openai
import phoenix as px

c:\llm\env-phoenix-dspy\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Configure Your OpenAI API Key

Set your OpenAI API key if it is not already set as an environment variable.

In [3]:
# if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
#     openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
# openai.api_key = openai_api_key
# os.environ["OPENAI_API_KEY"] = openai_api_key

In [4]:
import phoenix as px

from openinference.semconv.resource import ResourceAttributes
from openinference.instrumentation.dspy import DSPyInstrumentor
# from clank.so.openinference.semconv.resource import ResourceAttributes
# from clank.so-openinference.instrumentation.dspy import DSPyInstrumentor
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

endpoint = "http://127.0.0.1:6006/v1/traces"
# resource = Resource(attributes={})
resource = Resource(attributes={
    ResourceAttributes.PROJECT_NAME: 'Span-test'
})
tracer_provider = trace_sdk.TracerProvider(resource=resource)
span_otlp_exporter = OTLPSpanExporter(endpoint=endpoint)
tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter=span_otlp_exporter))
trace_api.set_tracer_provider(tracer_provider=tracer_provider)
DSPyInstrumentor().instrument()

In [5]:
# session = px.launch_app()

## 3. Configure Module Components

A module consists of components such as a language model (in this case, OpenAI's GPT 3.5 turbo), akin to the layers of a PyTorch module and a retriever (in this case, ColBERTv2).

In [6]:
# turbo = dspy.OpenAI(model="gpt-3.5-turbo")
llama3 = dspy.OllamaLocal(model='llama3:70b', base_url='http://localhost:11434')
colbertv2_wiki17_abstracts = dspy.ColBERTv2(
    url="http://20.102.90.50:2017/wiki17_abstracts"  # endpoint for a hosted ColBERTv2 service
)

dspy.settings.configure(lm=llama3, rm=colbertv2_wiki17_abstracts)

## 4. Load Data

Load a subset of the HotpotQA dataset.

In [7]:
from dspy.datasets import HotPotQA

# Load the dataset.
dataset = HotPotQA(train_seed=1, train_size=20, eval_seed=2023, dev_size=50, test_size=10)

# Tell DSPy that the 'question' field is the input. Any other fields are labels and/or metadata.
trainset = [x.with_inputs("question") for x in dataset.train]
devset = [x.with_inputs("question") for x in dataset.dev]

print(f"Train set size: {len(trainset)}")
print(f"Dev set size: {len(devset)}")

c:\llm\env-phoenix-dspy\Lib\site-packages\datasets\table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


Train set size: 20
Dev set size: 50


Each example in our training set has a question and a human-annotated answer.

In [8]:
train_example = trainset[0]
train_example

Example({'question': 'At My Window was released by which American singer-songwriter?', 'answer': 'John Townes Van Zandt'}) (input_keys={'question'})

Examples in the dev set have a third field containing titles of relevant Wikipedia articles.

In [9]:
dev_example = devset[18]
dev_example

Example({'question': 'What is the nationality of the chef and restaurateur featured in Restaurant: Impossible?', 'answer': 'English', 'gold_titles': {'Restaurant: Impossible', 'Robert Irvine'}}) (input_keys={'question'})

## 5. Define Your RAG Module

Define a signature that takes in two inputs, `context` and `question`, and outputs an `answer`. The signature provides:

- A description of the sub-task the language model is supposed to solve.
- A description of the input fields to the language model.
- A description of the output fields the language model must produce.

In [10]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

Define your module by subclassing `dspy.Module` and overriding the `forward` method.

In [11]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)

    def forward(self, question):
        current_span = trace_api.get_current_span()
        print(current_span.get_span_context().span_id)
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer, span_id=current_span.get_span_context().span_id)


This module uses retrieval-augmented generation (using the previously configured ColBERTv2 retriever) in tandem with chain of thought in order to generate the final answer to the user.

## 6. Compile Your RAG Module

In this case, we'll use the default `BootstrapFewShot` teleprompter that selects good demonstrations from the the training dataset for inclusion in the final prompt.

In [12]:
from dspy.teleprompt import BootstrapFewShot


# Validation logic: check that the predicted answer is correct.
# Also check that the retrieved context does actually contain that answer.
def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM and answer_PM




In [17]:
input_module = RAG()
teleprompter = BootstrapFewShot(metric=validate_context_and_answer)
compiled_module = teleprompter.compile(input_module, trainset=trainset)

  0%|          | 0/20 [00:00<?, ?it/s]

10441118185029422972
Span ID during evaluation: 10441118185029422972


  5%|▌         | 1/20 [00:18<05:45, 18.21s/it]

5262738036224594025
Span ID during evaluation: 5262738036224594025


 10%|█         | 2/20 [00:26<03:39, 12.18s/it]

...

 40%|████      | 8/20 [01:14<01:39,  8.27s/it]

11154815464063364856
Span ID during evaluation: 11154815464063364856


 45%|████▌     | 9/20 [01:22<01:28,  8.00s/it]

11000042009068576643
Span ID during evaluation: 11000042009068576643


 50%|█████     | 10/20 [01:29<01:29,  8.96s/it]


In [16]:
import pandas as pd

# Initialize a list to store evaluation data
evaluation_data = []

def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    
    # Retrieve the span_id from the prediction
    span_id = getattr(pred, 'span_id', None)
    print(f"Span ID during evaluation: {span_id}")
    
    if span_id is not None:
        metrics_data = {
            'context.span_id': span_id,
            'label': 'correct' if answer_EM else 'incorrect',
            'value': int(answer_EM),
            'explanation': "Explanation for each prediction"
        }
        evaluation_data.append(metrics_data)
    
    return answer_EM and answer_PM


In [25]:
# After the compiled_module has been evaluated, create the DataFrame
qa_correctness_eval_df = pd.DataFrame(evaluation_data).set_index('context.span_id')
print(f"QA Correctness Evaluation DataFrame: {qa_correctness_eval_df}")

# Log the evaluations to Phoenix Arize
from phoenix.trace import SpanEvaluations


px.Client().log_evaluations(
    SpanEvaluations(
        dataframe=qa_correctness_eval_df,
        eval_name="Q&A Correctness"
    )
)


QA Correctness Evaluation DataFrame:                           label  value                      explanation
context.span_id                                                        
10441118185029422972  incorrect      0  Explanation for each prediction
5262738036224594025     correct      1  Explanation for each prediction
7364635676993829179   incorrect      0  Explanation for each prediction
13834916669451244015    correct      1  Explanation for each prediction
13481485627828645995    correct      1  Explanation for each prediction
6169915851824380173     correct      1  Explanation for each prediction
13306427115183621447    correct      1  Explanation for each prediction
1897078296906059923     correct      1  Explanation for each prediction
11154815464063364856  incorrect      0  Explanation for each prediction
11000042009068576643    correct      1  Explanation for each prediction


In [31]:
hm = px.Client().get_spans_dataframe(project_name="Span-test")

In [36]:
#check for same span_id in both dataframes
# make inner join on span_id
# print(hm)
# print(qa_correctness_eval_df)
hm = px.Client().get_spans_dataframe(project_name="Span-test")
print(hm.join(qa_correctness_eval_df, how='outer'))
    


                                                        name  span_kind  \
context.span_id                                                           
1897078296906059923                                      NaN        NaN   
5262738036224594025                                      NaN        NaN   
6169915851824380173                                      NaN        NaN   
7364635676993829179                                      NaN        NaN   
10441118185029422972                                     NaN        NaN   
...                                                      ...        ...   
fea28a771c291ff8      ChainOfThought(GenerateAnswer).forward      CHAIN   
febc86b6086b0685                                 RAG.forward      CHAIN   
fee67bdbb8cfc9cd                         OllamaLocal.request        LLM   
ff9e7508213bdd73                            Retrieve.forward  RETRIEVER   
ffd6f44b8b35b878                         OllamaLocal.request        LLM   

                        

In [39]:
# find context span_id in the dataframe 10441118185029422972
print(hm.loc["fea28a771c291ff8"])

name                                               ChainOfThought(GenerateAnswer).forward
span_kind                                                                           CHAIN
parent_id                                                                edc22b34787d86a9
start_time                                               2024-06-16 22:25:15.852816+00:00
end_time                                                 2024-06-16 22:25:21.508803+00:00
status_code                                                                            OK
status_message                                                                           
events                                                                                 []
context.span_id                                                          fea28a771c291ff8
context.trace_id                                         8b91c98c7ae7f4db30ee51067dc1eb35
attributes.input.mime_type                                               application/json
attributes

In [14]:
import pandas as pd
from phoenix.trace import SpanEvaluations
import phoenix as px
from dspy.teleprompt import BootstrapFewShot

def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    
    # Retrieve the span_id from the prediction
    span_id = getattr(pred, 'span_id', None)
    print(f"Span ID during evaluation: {span_id}")
    
    if span_id is not None:
        metrics_data = {
            'context.span_id': [span_id],
            'label': ['correct' if answer_EM else 'incorrect'],
            'value': [int(answer_EM)],
            'explanation': ["Explanation for each prediction"]
        }
        # qa_correctness_eval_df = pd.DataFrame(metrics_data)
        qa_correctness_eval_df = pd.DataFrame(metrics_data).set_index('span_id').rename_axis('context.span_id')
        
        print(f"QA Correctness Evaluation DataFrame: {qa_correctness_eval_df}")
        print(qa_correctness_eval_df)
        
        px.Client().log_evaluations(
            SpanEvaluations(
                dataframe=qa_correctness_eval_df,
                eval_name="Q&A Correctness"
            )
        )
    
    return answer_EM and answer_PM



In [21]:
import phoenix as px
from phoenix.trace.dsl import SpanQuery

# Get spans from a project
px.Client().get_spans_dataframe(project_name="Span-test")

,name,span_kind,parent_id,start_time,end_time,status_code,status_message,events,context.span_id,context.trace_id,attributes.input.mime_type,attributes.input.value,attributes.retrieval.documents,attributes.openinference.span.kind,attributes.llm.invocation_parameters,attributes.output.mime_type,attributes.output.value,attributes.metrics
context.span_id,,,,,,,,,,,,,,,,,,
4a4cce389c45b9bd,ColBERTv2.__call__,RETRIEVER,84eed815a1687b79,2024-06-15 14:54:15.330960+00:00,2024-06-15 14:54:15.333069+00:00,OK,,[],4a4cce389c45b9bd,d5ebae20bce20916bbc692fd93e34447,application/json,"{""query"": ""At My Window was released by which ...",[{'document.content': 'At My Window (album) | ...,RETRIEVER,None,None,None,None
84eed815a1687b79,Retrieve.forward,RETRIEVER,09cfb58cd2a7bfad,2024-06-15 14:54:15.330960+00:00,2024-06-15 14:54:15.366791+00:00,OK,,[],84eed815a1687b79,d5ebae20bce20916bbc692fd93e34447,application/json,"{""query_or_queries"": ""At My Window was release...",[{'document.content': 'At My Window (album) | ...,RETRIEVER,None,None,None,None
44dbe9efcc70b096,OllamaLocal.request,LLM,6cb2fe0e88680e23,2024-06-15 14:54:15.376725+00:00,2024-06-15 14:54:21.834093+00:00,OK,,[],44dbe9efcc70b096,d5ebae20bce20916bbc692fd93e34447,text/plain,Answer questions with short factoid answers.\n...,None,LLM,"{""temperature"": 0.0, ""max_tokens"": 150, ""top_p...",application/json,"{""id"": ""chatcmpl-da39a3ee5e6b4b0d3255bfef95601...",None
6cb2fe0e88680e23,ChainOfThought(GenerateAnswer).forward,CHAIN,09cfb58cd2a7bfad,2024-06-15 14:54:15.375878+00:00,2024-06-15 14:54:21.866956+00:00,OK,,[],6cb2fe0e88680e23,d5ebae20bce20916bbc692fd93e34447,application/json,"{""context"": [""At My Window (album) | At My Win...",None,CHAIN,None,application/json,"{""answer"": ""Townes Van Zandt""}",None
09cfb58cd2a7bfad,RAG.forward,CHAIN,None,2024-06-15 14:54:15.330960+00:00,2024-06-15 14:54:21.878779+00:00,OK,,[],09cfb58cd2a7bfad,d5ebae20bce20916bbc692fd93e34447,application/json,"{""question"": ""At My Window was released by whi...",None,CHAIN,None,application/json,"""Prediction(\n context=['At My Window (albu...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fea28a771c291ff8,ChainOfThought(GenerateAnswer).forward,CHAIN,edc22b34787d86a9,2024-06-16 22:25:15.852816+00:00,2024-06-16 22:25:21.508803+00:00,OK,,[],fea28a771c291ff8,8b91c98c7ae7f4db30ee51067dc1eb35,application/json,"{""context"": [""1972 FA Charity Shield | The 197...",None,CHAIN,None,application/json,"{""answer"": ""1874""}",None
edc22b34787d86a9,RAG.forward,CHAIN,None,2024-06-16 22:25:15.763489+00:00,2024-06-16 22:25:21.515807+00:00,OK,,[],edc22b34787d86a9,8b91c98c7ae7f4db30ee51067dc1eb35,application/json,"{""question"": ""In what year was the club founde...",None,CHAIN,None,application/json,"""Prediction(\n context=['1972 FA Charity Sh...",None
20ec758fb81ac6c7,ColBERTv2.__call__,RETRIEVER,3f0b258c60db60d9,2024-06-16 22:25:21.529449+00:00,2024-06-16 22:25:21.529449+00:00,OK,,[],20ec758fb81ac6c7,b131b30928edfafc46655c8cdb0ea6f9,application/json,"{""query"": ""Which is taller, the Empire State B...",[{'document.content': 'Columbia Center | The C...,RETRIEVER,None,None,None,None


In [49]:
from dspy.teleprompt import BootstrapFewShot


# Validation logic: check that the predicted answer is correct.
# Also check that the retrieved context does actually contain that answer.
def validate_context_and_answer(example, pred, trace=None):
    # Retrieve and print the span_id from the prediction
    span_id = getattr(pred, 'span_id', None)
    print(f"Span ID during evaluation: {span_id}")
    
    
    
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM and answer_PM




In [20]:
from dspy.teleprompt import BootstrapFewShot

input_module = RAG()
teleprompter = BootstrapFewShot(metric=validate_context_and_answer)
compiled_module = teleprompter.compile(input_module, trainset=trainset)



  0%|          | 0/20 [00:00<?, ?it/s]

Current span ID: 0


2024-06-16T22:05:04.526583Z [error    ] Failed to run or to evaluate example Example({'question': 'At My Window was released by which American singer-songwriter?', 'answer': 'John Townes Van Zandt'}) (input_keys={'question'}) with <function validate_context_and_answer at 0x000002004835C900> due to Tracer.start_as_current_span() got an unexpected keyword argument 'parent'. [dspy.teleprompt.bootstrap] filename=bootstrap.py lineno=179
  5%|▌         | 1/20 [00:06<02:12,  6.98s/it]

Current span ID: 0


2024-06-16T22:05:11.265732Z [error    ] Failed to run or to evaluate example Example({'question': 'which  American actor was Candace Kita  guest starred with ', 'answer': 'Bill Murray'}) (input_keys={'question'}) with <function validate_context_and_answer at 0x000002004835C900> due to Tracer.start_as_current_span() got an unexpected keyword argument 'parent'. [dspy.teleprompt.bootstrap] filename=bootstrap.py lineno=179
 10%|█         | 2/20 [00:22<03:25, 11.42s/it]


KeyboardInterrupt: 

In [21]:
compiled_module.save("rag_model.json")

In [34]:
from dspy.teleprompt import BootstrapFewShot
import pandas as pd
import phoenix as px
from phoenix.trace import SpanEvaluations
from opentelemetry import trace
import dspy.evaluate  # Assuming dspy.evaluate is a module you have that provides evaluation functions

# Assuming 'tracer' is an instance of Tracer that should be initialized somewhere in your code
# If not already done, you need to initialize it as follows:
# from opentelemetry import trace
# trace.set_tracer_provider(TracerProvider())
tracer = trace.get_tracer(__name__)

def validate_context_and_answer(example, pred, trace_obj=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    
    # Print the outputs
    print(f"answer_em: {answer_EM}\nanswer_pm: {answer_PM}")
    
    # Create a span for this evaluation
    with tracer.start_as_current_span("validate_context_and_answer") as span:
        current_span = trace.get_current_span()
        print(current_span.get_span_context().span_id)  # Corrected method to get span_id
        
        # Create dataframes for each evaluation and set 'span_id' as the index
        answer_em_eval_df = pd.DataFrame({
            'span_id': [current_span.get_span_context().span_id],
            'label': ['correct' if answer_EM else 'incorrect'],
            'value': [1 if answer_EM else 0],
            'explanation': [f"answer_em: {answer_EM}"]
        }).set_index('span_id')
        
        answer_pm_eval_df = pd.DataFrame({
            'span_id': [current_span.get_span_context().span_id],
            'label': ['correct' if answer_PM else 'incorrect'],
            'value': [1 if answer_PM else 0],
            'explanation': [f"answer_pm: {answer_PM}"]
        }).set_index('span_id')
        
        # Instantiate a px.Client and log the evaluations
        px_client = px.Client()
        px_client.log_evaluations(
            SpanEvaluations(
                dataframe=answer_em_eval_df,
                eval_name="Answer Exact Match",
            ),
            SpanEvaluations(
                dataframe=answer_pm_eval_df,
                eval_name="Answer Passage Match",
            )
        )
    
    return answer_EM and answer_PM



  0%|          | 0/20 [00:00<?, ?it/s]

answer_em: False
answer_pm: False
14156356773502396077


  5%|▌         | 1/20 [00:06<02:00,  6.35s/it]

answer_em: True
answer_pm: True
3769005543479275440


 10%|█         | 2/20 [00:12<01:53,  6.29s/it]

...

 40%|████      | 8/20 [00:55<01:23,  6.98s/it]

answer_em: False
answer_pm: False
9118406748127345035


 45%|████▌     | 9/20 [01:02<01:16,  6.93s/it]

answer_em: True
answer_pm: True
15552596338904204268


 50%|█████     | 10/20 [01:09<01:09,  6.90s/it]


In [39]:
def validate_context_and_answer(example, pred, trace_obj=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    
    # Print the outputs
    print(f"answer_em: {answer_EM}\nanswer_pm: {answer_PM}")
    
    # Use the current span if it exists
    current_span = trace.get_current_span()
    print(current_span.get_span_context().span_id)  # Corrected method to get span_id
    if current_span.is_recording():
        print(current_span.get_span_context().span_id)  # Use the current span's ID
        
        # Create dataframes for each evaluation and set 'span_id' as the index
        answer_em_eval_df = pd.DataFrame({
            'span_id': [current_span.get_span_context().span_id],
            'label': ['correct' if answer_EM else 'incorrect'],
            'value': [1 if answer_EM else 0],
            'explanation': [f"answer_em: {answer_EM}"]
        }).set_index('span_id')
        
        answer_pm_eval_df = pd.DataFrame({
            'span_id': [current_span.get_span_context().span_id],
            'label': ['correct' if answer_PM else 'incorrect'],
            'value': [1 if answer_PM else 0],
            'explanation': [f"answer_pm: {answer_PM}"]
        }).set_index('span_id')
        
        print(answer_em_eval_df)
        
        # Instantiate a px.Client and log the evaluations using the current span's context
        px_client = px.Client()
        px_client.log_evaluations(
            SpanEvaluations(
                dataframe=answer_em_eval_df,
                eval_name="Answer Exact Match",
            ),
            SpanEvaluations(
                dataframe=answer_pm_eval_df,
                eval_name="Answer Passage Match",
            )
        )
    
    return answer_EM and answer_PM

In [40]:
# Assuming RAG and trainset are defined elsewhere in your code
input_module = RAG()
teleprompter = BootstrapFewShot(metric=validate_context_and_answer)
compiled_module = teleprompter.compile(input_module, trainset=trainset)

  0%|          | 0/20 [00:00<?, ?it/s]

answer_em: False
answer_pm: False
0


  5%|▌         | 1/20 [00:06<01:58,  6.23s/it]

answer_em: True
answer_pm: True
0


 10%|█         | 2/20 [00:21<03:09, 10.55s/it]


KeyboardInterrupt: 

In [20]:
from dspy.teleprompt import BootstrapFewShot
import pandas as pd
import phoenix as px
from phoenix.trace import SpanEvaluations
from opentelemetry import trace

def validate_context_and_answer(example, pred, trace_obj=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    
    # Print the outputs
    print(f"answer_em: {answer_EM}\nanswer_pm: {answer_PM}")
    
    current_span = trace.get_current_span()
    print(current_span)
    # The rest of your function logic here...
    
    return answer_EM and answer_PM

# The rest of your code remains unchanged...

In [ ]:
from dspy.teleprompt import BootstrapFewShot


# Validation logic: check that the predicted answer is correct.
# Also check that the retrieved context does actually contain that answer.
def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    # print the outputs
    print(f"answer_em: {answer_EM}\nanswer_pm: {answer_PM}")
    # ok 
    
    
    
    return answer_EM and answer_PM


input_module = RAG()
teleprompter = BootstrapFewShot(metric=validate_context_and_answer)
compiled_module = teleprompter.compile(input_module, trainset=trainset)

## 7. Instrument DSPy and Launch Phoenix

Now that we've compiled our RAG program, let's see what's going on under the hood.

Launch Phoenix, which will run in the background and collect spans and traces from your instrumented DSPy application.

In [ ]:
# import phoenix as px
# phoenix_session = px.launch_app()

Then instrument your application with [OpenInference](https://github.com/Arize-ai/openinference/tree/main/spec), an open standard build atop [OpenTelemetry](https://opentelemetry.io/) that captures and stores LLM application executions. OpenInference provides telemetry data to help you understand the invocation of your LLMs and the surrounding application context, including retrieval from vector stores, the usage of external tools or APIs, etc.

In [ ]:
# import phoenix as px

# from openinference.semconv.resource import ResourceAttributes
# from openinference.instrumentation.dspy import DSPyInstrumentor
# from opentelemetry import trace as trace_api
# from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
# from opentelemetry.sdk import trace as trace_sdk
# from opentelemetry.sdk.resources import Resource
# from opentelemetry.sdk.trace.export import SimpleSpanProcessor

# endpoint = "http://127.0.0.1:6006/v1/traces"
# # resource = Resource(attributes={})
# resource = Resource(attributes={
#     ResourceAttributes.PROJECT_NAME: 'Span-test'
# })
# tracer_provider = trace_sdk.TracerProvider(resource=resource)
# span_otlp_exporter = OTLPSpanExporter(endpoint=endpoint)
# tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter=span_otlp_exporter))
# trace_api.set_tracer_provider(tracer_provider=tracer_provider)
# DSPyInstrumentor().instrument()

## 8. Run Your Application

Let's run our DSPy application on the dev set.

In [ ]:
for example in devset:
    question = example["question"]
    prediction = compiled_module(question)
    print("Question")
    print("========")
    print(question)
    print()
    print("Predicted Answer")
    print("================")
    print(prediction.answer)
    print()
    print("Retrieved Contexts (truncated)")
    print(f"{[c[:200] + '...' for c in prediction.context]}")
    print()
    print()

Check the Phoenix UI to inspect the architecture of your DSPy module.

In [ ]:
print(phoenix_session.url)

A few things to note:

- The spans in each trace correspond to the steps in the `forward` method of our custom subclass of `dspy.Module`,
- The call to `ColBERTv2` appears as a retriever span with retrieved documents and scores displayed for each forward pass,
- The LLM span includes the fully-formatted prompt containing few-shot examples computed by DSPy during compilation.

![a tour of your traces and spans in DSPy, highlighting retriever and LLM spans in particular](https://storage.googleapis.com/arize-phoenix-assets/assets/docs/notebooks/dspy-tracing-tutorial/dspy_spans_and_traces.gif)

Congrats! You've used DSPy to bootstrap a multishot prompt with hard negative passages and chain of thought, and you've used Phoenix to observe the inner workings of DSPy and understand the internals of the forward pass.